In [12]:
import pandas as pd
import polars as pl

In [3]:
#Put a switch that does this
input_path = r"C:\Users\Mert\OneDrive\Desktop\Courses\Financial Big Data\data\nasdaq_exteral_data.csv"

In [4]:
def filter_and_save_parquet(input_path: str, ticker_list):
    """
    Filters data from a CSV file, processes it as per the requirements,
    and writes the filtered data back to respective Parquet files.

    Parameters:
        input_path (str): The file path to the input CSV file.
        ticker_list (list): A list of stock tickers to filter (e.g., ["AAPL", "GOOG", "TSLA"]).

    Returns:
        None
    """
    # Process each ticker separately to reduce memory usage
    for ticker in ticker_list:
        # Read and filter data in streaming mode
        lazy_df = pl.scan_csv(input_path)
        filtered_df = (
            lazy_df
            .select(["Date", "Stock_symbol", "Article_title"])  # Select specific columns
            .filter(pl.col("Article_title").is_not_null())  # Filter out null values
            .filter(pl.col("Stock_symbol") == ticker)  # Filter rows matching the ticker
            .with_columns(
                pl.col("Date").str.to_datetime("%Y-%m-%d %H:%M:%S UTC")  # Convert Date column to datetime
            )
        )

        # Collect filtered data and write directly to a Parquet file
        output_path = f"../raw/data/{ticker}_news.parquet"
        filtered_df.sink_parquet(output_path)  # Stream writing without converting to Pandas


In [ ]:
#Applying the function (can extract as many tickers you like)
filter_and_save_parquet(input_path = input_path, ticker_list = ["AAPL","GOOG","TSLA"])

In [32]:
import yfinance as yf
apple_data = yf.download('AAPL', interval='1h', start='2023-01-01')
apple_data


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-03 09:30:00-05:00,130.279999,130.899994,125.230003,125.459999,125.459999,27775505
2023-01-03 10:30:00-05:00,125.459999,125.870003,124.730003,125.345001,125.345001,18104999
2023-01-03 11:30:00-05:00,125.350197,125.370003,124.349098,124.660004,124.660004,11248777
2023-01-03 12:30:00-05:00,124.660004,124.940002,124.190002,124.649902,124.649902,8860080
2023-01-03 13:30:00-05:00,124.669899,125.000000,124.190002,124.570000,124.570000,8388062
...,...,...,...,...,...,...
2024-12-26 11:30:00-05:00,258.890015,259.339996,258.820007,258.984985,258.984985,1989211
2024-12-26 12:30:00-05:00,258.988007,259.325012,258.570007,258.818909,258.818909,1857025
2024-12-26 13:30:00-05:00,258.809998,258.948914,258.410004,258.600006,258.600006,1532082
